## Weather Data : Initial Visualization

### Delaware

In [1]:
state="DE"

In [2]:
#sc.stop()

In [3]:
import pandas as pd
import numpy as np
import sklearn as sk
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
%%time
%run lib/startup.py

172.17.0.2


FileNotFoundError: [Errno 2] No such file or directory: 'environ.yaml'

CPU times: user 399 ms, sys: 21.9 ms, total: 421 ms
Wall time: 431 ms


In [5]:
sc

''

In [6]:
import warnings  # Suppress Warnings
warnings.filterwarnings('ignore')

_figsize=(10,7)

## Read Data

### show 3 rows of joined weather+stations table

In [7]:
### Total number of stations
stations.count()

NameError: name 'stations' is not defined

In [ ]:
%%time
weather=measurements.join(stations,on='station')
weather.show(3)

In [ ]:
sqlContext.registerDataFrameAsTable(weather,'weather')

In [ ]:
ms=['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
# ms=['TMAX', 'TMIN', 'TOBS']
cms='or\n'.join(['Measurement="%s" '%(m) for m in ms])

##  read all data for state
Query="""
SELECT *
FROM weather
WHERE state="%s" and 
(%s)
"""%(state,cms)
print(Query)

In [ ]:
%%time
weather_df=sqlContext.sql(Query)
print('number of rows in result=',weather_df.count())
weather_df.show(2)

### read or compute statistics information for state.

In [ ]:
%%time
import os.path
from lib.computeStatistics import computeStatistics
from pickle import dump,load

pkl_filename=parquet_root+'weather-statistics/'+state+'-'+','.join(ms)+'.pkl'
print(pkl_filename)
!ls $pkl_filename

In [ ]:
%%time
if os.path.isfile(pkl_filename):   
    print('precomputed statistics file exists')
    with open(pkl_filename,'br') as pkl_file:
        STAT=load(pkl_file)
else:
    print('computing statistics')
    STAT=computeStatistics(sqlContext,weather_df,measurements=ms)
    with open(pkl_filename,'bw') as pkl_file:
        dump(STAT,pkl_file)

STAT.keys()

In [ ]:
print("   Name  \t                 Description             \t  Size")
print("-"*80)
print('\n'.join(["%10s\t%40s\t%s"%(s[0],s[1],str(s[2])) for s in STAT_Descriptions]))

### print statistics for TOBS

In [ ]:
S=STAT['TOBS']
for key in ['mean', 'std', 'low100', 'high100']:
    element=S[key]
    print(key,'=',end='')
    if type(element)==numpy.float64 or type(element)==numpy.float16:
        print('%6.2f'%element)
    elif type(element)==numpy.ndarray:
        print (element)
    else:
        print('unidentified type=',type(element))

In [ ]:
%pylab inline
measurement='TOBS'
Sobs=STAT[measurement]['SortedVals']

#figure(figsize=[15,10])
n_obs=Sobs.shape[0]
p=arange(0,1,1/n_obs)
plot(Sobs,p)
title('CDF of '+measurement)
grid()

### distribution of undefined elements in yearly measurements.

In [ ]:
hist(S['UnDef'],bins=100);
xlabel('number of undefined days in a year');
title(state+'     '+measurement);

In [ ]:
%%time
## Find the number of measurements of each of the common 6 types in MA

Query="""
select Measurement,count(Measurement) as count
from weather
WHERE state='%s'
GROUP BY Measurement
ORDER BY count DESC
"""%state
sqlContext.sql(Query).show(15)

### Find out the definition of measurement types

In [ ]:
!grep SNWD $parquet_root/noaa/readme.txt

In [ ]:
!grep MDPR $parquet_root/noaa/readme.txt

## Find the number of measurements (years) for each station in state

In [ ]:
Query="""
select Station,count(Station) as count
from weather
WHERE state='%s' and Measurement='TOBS'
GROUP BY Station
ORDER BY count DESC
"""%state
#print(Query)
tmp=sqlContext.sql(Query)
print(tmp.count())
tmp.show(3)

### show details of state

In [ ]:
station="USC00072730"
Query="""
select * from stations
WHERE Station="%s"
"""%station
sqlContext.sql(Query).show()

### find how many measurements of each type for a particlar station

In [ ]:
Query="""
SELECT Measurement,count(Measurement) as count 
FROM weather
WHERE Station='%s' 
GROUP BY Measurement
ORDER BY count DESC
"""%station
sqlContext.sql(Query).show(8)

### find year with all measurements

In [ ]:
%%time
#find year with all measurements
Query="""
SELECT Year,count(Year) as count 
FROM weather
WHERE Station='%s' 
GROUP BY Year
ORDER BY count DESC
"""%station
pdf=sqlContext.sql(Query).toPandas()

pdf.head()

In [ ]:
from collections import Counter
Counter(list(pdf['count']))

### get all measurements for year=1960, measurement="TOBS"  and state="DE"

In [ ]:
year=1990
Query="""
SELECT *
FROM weather
WHERE Year='%s' 
and Measurement='TOBS'
and State='%s'
"""%(year,state)
df=sqlContext.sql(Query)
print(df.count())
df.show(3)

In [ ]:
%%time

## Collect measurements for selected rows
ListOfRows=df.rdd.map(lambda x: (x.Station,unpackAndScale(x))).collect()
_labels = [x[0] for x in ListOfRows]
M=stack([x[1] for x in ListOfRows])
M.shape
        

### Plot all 35 the TOBS measurements

In [ ]:
from lib.YearPlotter import YearPlotter
k=5
_title='TOBS for %s / %d'%(station,year)
fig, ax = plt.subplots(figsize=_figsize);
YP=YearPlotter()
YP.plot(M[:k,:366].T,fig,ax,title=_title,labels=_labels)# ,labels=labels);
ylabel('temp in centigrade');

### Plot all 35 TOBS measurements

In [ ]:
#looking at all of the year creates a mess
from lib.YearPlotter import YearPlotter
k=35
fig, ax = plt.subplots(figsize=_figsize);
YP=YearPlotter()
YP.plot(M[:k,:366].T,fig,ax,title=_title,labels=_labels)# ,labels=labels);
ylabel('temp in centigrade');

### Zooming into a small number of days the correlations between stations are clear.

In [ ]:
## Looking at a small range of days  across all stations in state reveals 
## Interesting clustering
figure(figsize=_figsize)
plot(M[:,70:95].transpose());
grid()

In [ ]:
# get all measurements for a particular year and a particular station
Query="""
SELECT *
FROM weather
WHERE Station='%s' 
and Year=%d
"""%(station,year )
print(Query)
pandas_df=sqlContext.sql(Query).toPandas()
pandas_df=pandas_df.set_index('Measurement')
print('pandas_df.shape=',pandas_df.shape)
print('pandas_df.columns=',pandas_df.columns)

In [ ]:
_tmax=unpackArray(pandas_df.loc['TMAX','Values'],np.int16)/10.
_tmin=unpackArray(pandas_df.loc['TMIN','Values'],np.int16)/10.
_tobs=unpackArray(pandas_df.loc['TOBS','Values'],np.int16)/10.
figure(figsize=_figsize)
plot(_tmax,label='TMAX');
plot(_tmin,label='TMIN');
plot(_tobs,label='TOBS');

xlabel('day of year')
ylabel('degrees centigade')
title('Temperatures for %s in %d'%(station,year))
legend()
grid()

### Script for plotting yearly plots

In [ ]:
from lib.YearPlotter import YearPlotter
T=np.stack([_tmin,_tmax,_tobs])

fig, ax = plt.subplots(figsize=_figsize);
YP=YearPlotter()
YP.plot(T.transpose(),fig,ax,title='Temperatures for %s in %d'%(station,year));
plt.savefig('percipitation.png')
#title('A sample of graphs');

## Statistics across the state

### Distribution of missing observations
The distribution of missing observations is not uniform throughout the year. We visualize it below.

In [ ]:
from lib.MultiPlot import *
YP=YearPlotter()
def plot_valid(m,fig,axis):
    valid_m=STAT[m]['NE']
    YP.plot(valid_m,fig,axis,title='valid-counts '+m,label=m)
    

In [ ]:
plot_pair(['TMIN','TMAX'],plot_valid)

In [ ]:
plot_pair(['TOBS','PRCP'],plot_valid)

In [ ]:
# Note that for "SNOW" there are more missing measurements in the summer
# While for SNWD there are less missing in the summer
# Question: do these anomalies involve the same stations?
plot_pair(['SNOW', 'SNWD'],plot_valid)

### Plots of mean and std of observations

In [ ]:
def plot_mean_std(m,fig,axis):
    scale=1.
    temps=['TMIN','TMAX','TOBS']
    percipitation=['PRCP','SNOW','SNWD']
    _labels=['mean+std','mean','mean-std']
    if (m in percipitation):
        scale=10.
    mean=STAT[m]['Mean']/scale
    std=np.sqrt(STAT[m]['Var'])/scale
    graphs=np.vstack([mean+std,mean,mean-std]).transpose()
    YP.plot(graphs,fig,axis,labels=_labels,title='Mean+-std   '+m)
    if (m in temps):
        axis.set_ylabel('Degrees Celsius')
    if (m in percipitation):
        axis.set_ylabel('millimeter')



In [ ]:
plot_pair(['TMIN','TMAX'],plot_mean_std)

In [ ]:
plot_pair(['TMAX','TOBS'],plot_mean_std)

In [ ]:
plot_pair(['PRCP','SNOW'],plot_mean_std)

In [ ]:
plot_pair(['SNOW', 'SNWD'],plot_mean_std)

## Conclusion

* We loaded the weather data from Parquet files.
* We explored statistics for the data.
* We explored where there are a lot of empty cells - limits the accuracy of the statistics.
* We visualized different measurements as a function of the day of the year.
* **next** Using PCA for more refined analysis.